In [245]:
import pylogit as pl
import pandas as pd
import numpy as np
import json
import time
from collections import OrderedDict

from transport_network import approx_shape_centroid, get_haversine_distance, Polygon_Location

In [72]:
state_codes={'Detroit': 'mi', 'Boston': 'ma'}
state_fips={'Detroit': '26', 'Boston': '25'}
NUM_ALTS=8
sample_size=5000

PUMA_POP_PATH='./cities/'+'Detroit'+'/raw/ACS/population.csv'
# https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/
PUMS_HH_PATH='./cities/'+'Detroit'+'/raw/PUMS/csv_h'+state_codes['Detroit']+'/ss16h'+state_codes['Detroit']+'.csv'
PUMS_POP_PATH='./cities/'+'Detroit'+'/raw/PUMS/csv_p'+state_codes['Detroit']+'/ss16p'+state_codes['Detroit']+'.csv'
#        POI_PATH = './cities/'+self.city_folder+'/raw/OSM/poi.geojson'
PUMA_TO_POW_PUMA_PATH='./puma_to_pow_puma.csv'
PUMA_SHAPE_PATH = './cities/'+'Detroit'+'/raw/pumas.geojson'
PUMAS_INCLUDED_PATH = './cities/'+'Detroit'+'/raw/pumas_included.json'

In [229]:
hh=pd.read_csv(PUMS_HH_PATH)
pop = pd.read_csv(PUMS_POP_PATH)
hh['PUMA']=hh.apply(lambda row: str(int(row['PUMA'])).zfill(5), axis=1)
pop['PUMA']=pop.apply(lambda row: str(int(row['PUMA'])).zfill(5), axis=1)
pop['POWPUMA']=pop.apply(lambda row: str(int(row['POWPUMA'])).zfill(5) 
                        if not np.isnan(row['POWPUMA']) else 'NaN', axis=1)

#        all_PUMAs=list(set(hh['PUMA']))
pumas_included=json.load(open(PUMAS_INCLUDED_PATH))                                         # For the whole MI
pumas_shape=json.load(open(PUMA_SHAPE_PATH))
pumas_order=[f['properties']['PUMACE10'] for f in pumas_shape['features']]
            
puma_pop = pd.read_csv(PUMA_POP_PATH)
puma_pop = puma_pop.loc[puma_pop['STATE']==int(state_fips['Detroit'])]
puma_pop['PUMA']=puma_pop.apply(lambda row: str(row['PUMA']).zfill(5), axis=1)
puma_pop=puma_pop.set_index('PUMA')


# identify recent movers and vacant houses                                            
hh_vacant_for_rent=hh[(hh['VACS']==1) & (hh['PUMA'].isin(pumas_included['puma']))].copy()          
hh_rented=hh[(hh['TEN']==3) & (hh['PUMA'].isin(pumas_included['puma']))].copy()                                                      
renters_recent_move=hh_rented[hh_rented['MV']==1].copy()    

# get the area of each PUMA
puma_land_sqm=dict(zip(puma_pop.index, puma_pop['AREALAND'].astype('int64')))


In [230]:
# get the distance between each puma and each pow-puma
pow_puma_df=pd.read_csv(PUMA_TO_POW_PUMA_PATH, skiprows=1, header=1)
pow_puma_df_state=pow_puma_df.loc[pow_puma_df[
        'State of Residence (ST)']==state_fips['Detroit']].copy()
pow_puma_df_state['POW_PUMA']=pow_puma_df_state.apply(
        lambda row: str(int(row['PWPUMA00 or MIGPUMA1'])).zfill(5), axis=1)
pow_puma_df_state['PUMA']=pow_puma_df_state.apply(
        lambda row: str(int(row['PUMA'])).zfill(5), axis=1)
all_pow_pumas=set(pow_puma_df_state['POW_PUMA'])
pow_puma_to_puma={}
for p in all_pow_pumas:
    pow_puma_to_puma[p]=list(pow_puma_df_state.loc[
            pow_puma_df_state['POW_PUMA']==p, 'PUMA'].values)

In [231]:
# find the centroid of each puma
puma_centroids={}
pow_puma_centroids={}
for puma in set(pow_puma_df_state['PUMA']):
    centr=approx_shape_centroid(pumas_shape['features'][pumas_order.index(puma)]['geometry'])
    puma_centroids[puma]=centr

In [232]:
# find the centroid of each pow-puma
all_pow_pumas=set(pow_puma_df_state['POW_PUMA'])

for pow_puma in all_pow_pumas:
    pumas=pow_puma_to_puma[pow_puma]
    puma_centr=[puma_centroids[puma] for puma in pumas]
    # TODO, shold be weighted by area- ok if similar size
    pow_puma_centroids[pow_puma]=[np.mean([pc[0] for pc in puma_centr]),
                                    np.mean([pc[1] for pc in puma_centr])]

In [233]:
# calculate distance between puma and pow-puma
dist_mat={}
for puma in puma_centroids:
    dist_mat[puma]={}
    for pow_puma in pow_puma_centroids:
        dist = get_haversine_distance(
                puma_centroids[puma], pow_puma_centroids[pow_puma])
        # external distance
        if dist > 0:
            dist_mat[puma][pow_puma] = dist
        # inner distance
        else:
            dist_mat[puma][pow_puma] = np.sqrt(puma_land_sqm[puma] / np.pi)

In [234]:
# build the PUMA aggregate data data frame
median_income_by_puma=hh.groupby('PUMA')['HINCP'].median()
#TODO: get more zonal attributes such as access to employment, amenities etc.

puma_obj=[{'PUMA':puma,
            'med_income':median_income_by_puma.loc[puma],
            'puma_pop_per_sqm':float(puma_pop.loc[puma]['POP100'])/puma_land_sqm[puma]
            } for puma in pumas_included['puma']]

puma_attr_df=pd.DataFrame(puma_obj)
puma_attr_df=puma_attr_df.set_index('PUMA')

In [235]:
# create features at property level
# normalise rent stratifying by bedroom number
renters_recent_move.loc[renters_recent_move['BDSP']>2, 'BDSP']=3            # change [the number of bedroom] >2 to 3
renters_recent_move.loc[renters_recent_move['BDSP']<1, 'BDSP']=1            # change [the number of bedroom] <1 to 1
hh_vacant_for_rent.loc[hh_vacant_for_rent['BDSP']>2, 'BDSP']=3          
hh_vacant_for_rent.loc[hh_vacant_for_rent['BDSP']<1, 'BDSP']=1
rent_mean={}
rent_std={}
for beds in range(1,4):
    rent_mean[beds]=renters_recent_move.loc[renters_recent_move['BDSP']==beds, 'RNTP'].mean()
    rent_std[beds]=renters_recent_move.loc[renters_recent_move['BDSP']==beds, 'RNTP'].std()

In [236]:
for df in [renters_recent_move, hh_vacant_for_rent]:
    df['norm_rent']=df.apply(
        lambda row: (row['RNTP']-rent_mean[row['BDSP']])/rent_std[row['BDSP']], axis=1)
    # Age of building
    df['built_since_jan2010']=df.apply(lambda row: row['YBL']>=14, axis=1)
    df['puma_pop_per_sqmeter']=df.apply(lambda row: puma_attr_df.loc[row['PUMA']]['puma_pop_per_sqm'], axis=1)
    df['med_income']=df.apply(lambda row: puma_attr_df.loc[row['PUMA']]['med_income'], axis=1)  
all_rooms_available = pd.concat([hh_vacant_for_rent, renters_recent_move], axis=0) 
median_norm_rent = all_rooms_available.groupby('PUMA')['norm_rent'].median()
puma_attr_df['media_norm_rent'] =  puma_attr_df.apply(lambda row: median_norm_rent[row.name], axis=1)

In [237]:
# num of avaiable housing units in each PUMA
num_available_houses_in_puma = hh_vacant_for_rent.groupby('PUMA')['SERIALNO'].count()
pumas_included_pd = pd.Series(0, index = pumas_included['puma'])
num_available_houses_in_puma = (num_available_houses_in_puma + pumas_included_pd).fillna(0)
puma_attr_df['num_houses'] = puma_attr_df.apply(lambda row: num_available_houses_in_puma[row.name], axis=1)

renters_recent_move=renters_recent_move[['SERIALNO', 'PUMA','HINCP',  'norm_rent', 'RNTP', 'built_since_jan2010', 'puma_pop_per_sqmeter', 'med_income', 'BDSP', 'NP']]
hh_vacant_for_rent=hh_vacant_for_rent[['PUMA', 'HINCP', 'norm_rent', 'RNTP','built_since_jan2010', 'puma_pop_per_sqmeter', 'med_income', 'BDSP']]
    
rent_normalisation={"mean": rent_mean, "std": rent_std}

home_loc_mnl = {'home_loc_mnl_PUMAs': {}, 'home_loc_mnl_hh': {}}    

In [243]:
# Model Estimation
# First stage: choice model on PUMA level
# =============================================================================
long_data_PUMA = pd.DataFrame()
print('\n\n[info] Preparing long data for PUMA-level choice.')
time1 = time.time()
numPUMAs = puma_attr_df.shape[0]
ind = 0
for ind_actual, row_actual in renters_recent_move.iterrows():
    if ind >= sample_size:
        break
householdID = row_actual['SERIALNO']
places_of_work = set(pop.loc[pop['SERIALNO']==householdID, 'POWPUMA'])
places_of_work = [x for x in places_of_work if x in all_pow_pumas]
if len(places_of_work):
    this_sample_puma_attr_df = puma_attr_df.copy()
    this_sample_puma_attr_df['custom_id'] = ind_actual * np.ones(numPUMAs, dtype=np.int8)
    this_sample_puma_attr_df['choice_id'] = list(puma_attr_df.index)
    this_sample_puma_attr_df['choice'] = np.zeros(numPUMAs)
    this_sample_puma_attr_df['hh_income'] = row_actual['HINCP']
    this_sample_puma_attr_df.loc[this_sample_puma_attr_df['choice_id']==row_actual['PUMA'], 'choice'] = 1
    this_sample_puma_attr_df['work_dist'] = [np.mean([dist_mat[puma][pow_puma] 
                                            for pow_puma in places_of_work]) for puma in list(puma_attr_df.index)]
    long_data_PUMA = pd.concat([long_data_PUMA, this_sample_puma_attr_df], axis=0)
    ind += 1

long_data_PUMA['income_disparity']=long_data_PUMA.apply(lambda row: np.abs(row['hh_income']-row['med_income']), axis=1)
time2 = time.time()
print('[info] Long data for PUMA-level choice finished. Elapsed time: {} seconds'.format(time2-time1))



[info] Preparing long data for PUMA-level choice.
[info] Long data for PUMA-level choice finished. Elapsed time: 0.0575108528137207 seconds


In [248]:
long_data_PUMA.to_csv('./cities/'+'Detroit'+'/clean/logit_data_long_form/logit_data_PUMA.csv', index=False)


In [249]:
choiceModelPUMA_spec = OrderedDict()
choiceModelPUMA_names = OrderedDict()
choiceModelPUMAsRegressors = ['puma_pop_per_sqm', 'income_disparity', 'work_dist', 'media_norm_rent', 'num_houses'] + [x for x in list(long_data_PUMA.columns) if x.endswith('_den')]
for var in choiceModelPUMAsRegressors:
    choiceModelPUMA_spec[var] = [list(set(long_data_PUMA['choice_id']))]
    choiceModelPUMA_names[var] = [var]

home_loc_mnl_PUMAs = pl.create_choice_model(data=long_data_PUMA,
                                    alt_id_col='choice_id',
                                    obs_id_col='custom_id',
                                    choice_col='choice',
                                    specification=choiceModelPUMA_spec,
                                    model_type="MNL",
                                    names=choiceModelPUMA_names)
print('\n[info] Fitting Upper Level Model')
numCoef=sum([len(choiceModelPUMA_spec[s]) for s in choiceModelPUMA_spec])

# pylogit may encounter memory error in calculating Hessiann matrix for S.E. in this model, if so, switch to noHessian approach and only do point estimation.
try:
    home_loc_mnl_PUMAs.fit_mle(np.zeros(numCoef))
    print(home_loc_mnl_PUMAs.get_statsmodels_summary())
    home_loc_mnl['home_loc_mnl_PUMAs'] = {'just_point': False, 'model': home_loc_mnl_PUMAs}
except:
    home_loc_mnl_PUMAs_result = home_loc_mnl_PUMAs.fit_mle(np.zeros(numCoef), just_point=True)
    params = home_loc_mnl_PUMAs_result['x']
    print('\nLogit model parameters:\n---------------------------')
    for varname, para in zip(home_loc_mnl_PUMAs.ind_var_names, params):
        print('{}: {:4.6f}'.format(varname, para))
    home_loc_mnl['home_loc_mnl_PUMAs'] = {'just_point': True, 'model': home_loc_mnl_PUMAs, 'params': params, 'var_names': home_loc_mnl_PUMAs.ind_var_names}


[info] Fitting Upper Level Model
Log-likelihood at zero: -3.4965
Initial Log-likelihood: -3.4965


Estimation Time for Point Estimation: 0.05 seconds.
Final log-likelihood: -0.0000
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:                    1
Model:             Multinomial Logit Model   Df Residuals:                       -4
Method:                                MLE   Df Model:                            5
Date:                     Sat, 26 Aug 2023   Pseudo R-squ.:                   1.000
Time:                             12:12:16   Pseudo R-bar-squ.:              -0.430
AIC:                                10.000   Log-Likelihood:                 -0.000
BIC:                                 0.000   LL-Null:                        -3.497
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
puma_pop_per_sqm  1.981e+05   1.92e+10   1.03e-05      1.000   -3.77e+10    

c:\Users\Jiang\anaconda3\envs\cusplondon\lib\site-packages\scipy\optimize\_minimize.py:523: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)
c:\Users\Jiang\anaconda3\envs\cusplondon\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),
